# Задание из четвёртой лабы

Обучите один из алгоритмов классификации из библиотеки sklearn. Объяснять принцип работы классификатора в этой лабораторной не требуется

1. Выберите один из датасетов первой лабораторной


2. Найдите в данных признак, который будете предсказывать
    
    Если это категория, то у неё должно быть от 2 до 10 значений (например колонка species в датасете миграции птиц содержит 3 категории)
    
    Если признак количественный, то его нужно дискретизировать (например для популярности трека в датасете spotify можно разделить треки на 3 категории: 0-20 - популярные, 21-50 - обычные, 51 - 100 - непопулярные). Полученную категорию записать в датафрейм как отдельную колонку


3. Выделите признаки для предсказания
    
    Используйте от 2 до 5 количественных признаков для предсказания целевой переменной


4. Проверьте нормальность количественных признаков
    
    Используйте статистический тест из лекции. Если распределение является лог-нормальным, логарифмируйте его и проверьте на нормальность
    
    Как минимум 1 из признаков должен быть нормальным


5. Покажите качество предсказания модели с помощью метрик

    Метрика должна учитывать баланс классов в датасете

# 1. Датасет Spotify

In [126]:
from sklearn.ensemble import AdaBoostClassifier
from sklearn.metrics import precision_recall_curve, recall_score, precision_score, f1_score
import pandas as pd
from copy import copy
import numpy as np
import statistics as st
from scipy import stats

In [127]:
ds = pd.read_csv('data/SpotifyFeatures_2.csv')
ds.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,0.611,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.246,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,0.952,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,0.703,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,0.950,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390


# 2. Будем предсказывать категорию danceability aka танцевальность

In [128]:
split_ratio = 0.7
split_idx = int(split_ratio * len(ds))

ds.loc[ds["danceability"] < 0.3, "danceability_category"] = 1
ds.loc[ds["danceability"] >= 0.3, "danceability_category"] = 2
ds.loc[ds["danceability"] >= 0.6, "danceability_category"] = 3

d = copy(ds).drop(["genre", "artist_name", "track_name", "track_id", "key", "mode", "time_signature", "acousticness"], axis=1).values.tolist()

d_train = d[split_idx:]
d_value = d[:split_idx]

# 3. Выделим признаки для предсказания

In [129]:
ds.loc[ds['acousticness'] < 0.5, 'acousticness'] = 0
ds.loc[ds['acousticness'] >= 0.5, 'acousticness'] = 1

a = ds["acousticness"].values.tolist()

a_train = a[split_idx:]
a_value = a[:split_idx]
ds.head()

,genre,artist_name,track_name,track_id,popularity,acousticness,danceability,duration_ms,energy,instrumentalness,key,liveness,loudness,mode,speechiness,tempo,time_signature,valence,danceability_category
0,Movie,Henri Salvador,C'est beau de faire un Show,0BRjO6ga9RKCKjfDqeFgWV,0,1.0,0.389,99373,0.910,0.000,C#,0.3460,-1.828,Major,0.0525,166.969,4/4,0.814,2.0
1,Movie,Martin & les fées,Perdu d'avance (par Gad Elmaleh),0BjC1NfoEOOusryehmNudP,1,0.0,0.590,137373,0.737,0.000,F#,0.1510,-5.559,Minor,0.0868,174.003,4/4,0.816,2.0
2,Movie,Joseph Williams,Don't Let Me Be Lonely Tonight,0CoSDzoNIKCRs124s9uTVy,3,1.0,0.663,170267,0.131,0.000,C,0.1030,-13.879,Minor,0.0362,99.488,5/4,0.368,3.0
3,Movie,Henri Salvador,Dis-moi Monsieur Gordon Cooper,0Gc6TVm52BwZD07Ki6tIvf,0,1.0,0.240,152427,0.326,0.000,C#,0.0985,-12.178,Major,0.0395,171.758,4/4,0.227,1.0
4,Movie,Fabien Nataf,Ouverture,0IuslXpMROHdEPvSl1fTQK,4,1.0,0.331,82625,0.225,0.123,F,0.2020,-21.150,Major,0.0456,140.576,4/4,0.390,2.0


In [134]:
ada_boost = AdaBoostClassifier()
ada_boost.fit(d_train, a_train)
ada_boost.score(d_value, a_value)

0.8756591183927026

# 4. Статический тест

In [135]:
ds["normalized_column"] = np.random.normal(size=len(ds))
stats.normaltest(ds['normalized_column'])

NormaltestResult(statistic=1.0832023775828996, pvalue=0.5818159090321013)

# 5. Покажите качество предсказания модели с помощью метрик

In [136]:
a_predict = ada_boost.predict(d_value)

precision = precision_score(a_value, a_predict) # tp / (tp + fp)
recall = recall_score(a_value, a_predict) # tp / (tp + fn)
f1 = f1_score(a_value, a_predict)

print(f'Precision: {precision}, Recall: {recall}, F1: {f1}')

Precision: 0.8263389146182164, Recall: 0.7288476443848587, F1: 0.7745375214265043
